What about `XGBoost` instead of `RandomForest`?

In [ ]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import make_scorer
from kaggle.data import load_data, save_submission
from kaggle.metrics import rmse

In [ ]:
y, X, X_submission, ids_submission = load_data()
y = np.log(y)

In [9]:
parameters = {'max_depth': [5,7,9,11,13,15],
              'learning_rate': [0.5, 0.1, 0.2, 0.3],
              'n_estimators': [100],
              'min_child_weight': [1,2,4,8],
              'subsample': [0.4, 0.6, 0.8],
              'colsample_bytree': [0.4, 0.6, 0.8]}

model = RandomizedSearchCV(XGBRegressor(silent = True, objective='reg:linear', nthread=4, seed = 1337),
                           parameters, 
                           n_iter=60, 
                           random_state=1773,
                           scoring=make_scorer(rmse, greater_is_better=False), 
                           cv=KFold(5, random_state=1773))

In [10]:
%%time
model.fit(X,y)

CPU times: user 2min 36s, sys: 334 ms, total: 2min 37s
Wall time: 39.4 s


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1773, shuffle=False),
          error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1337, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=60, n_jobs=1,
          param_distributions={'colsample_bytree': [0.4, 0.6, 0.8], 'learning_rate': [0.5, 0.1, 0.2, 0.3], 'min_child_weight': [1, 2, 4, 8], 'n_estimators': [100], 'subsample': [0.4, 0.6, 0.8], 'max_depth': [5, 7, 9, 11, 13, 15]},
          pre_dispatch='2*n_jobs', random_state=1773, refit=True,
          return_train_score=True,
          scoring=make_scorer(rmse, greater_is_better=False), verbose=0)

In [11]:
print 'best score:', model.best_score_
print 'best parameters:', model.best_params_

best score: -0.129123947223
best parameters: {'colsample_bytree': 0.4, 'learning_rate': 0.1, 'min_child_weight': 4, 'n_estimators': 100, 'subsample': 0.8, 'max_depth': 7}


In [12]:
y_pred = model.predict(X_submission)
save_submission(ids_submission, y_pred, '0_random_search_shifted_xgboost')